# 학습 데이터 다운로드 및 전처리

## 학습 데이터 다운로드

In [10]:
import os
import shutil
import zipfile

import pandas as pd
import tensorflow as tf
import urllib3
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

필요한 Parallel corpus 파일 다운로드

In [11]:
# import requests

# headers = {
#     'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
# }

# def download_zip(url, output_path):
#     response = requests.get(url, headers=headers, stream=True)
#     if response.status_code == 200:
#         with open(output_path, 'wb') as f:
#             for chunk in response.iter_content(chunk_size=8192):
#                 f.write(chunk)
#         print(f"ZIP file downloaded to {output_path}")
#     else:
#         print(f"Failed to download. HTTP Response Code: {response.status_code}")

# url = "http://www.manythings.org/anki/fra-eng.zip"
# output_path = "fra-eng.zip"
# download_zip(url, output_path)

# path = os.getcwd()
# zipfilename = os.path.join(path, output_path)

# with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
#     zip_ref.extractall(path)


In [12]:
lines = pd.read_csv('fra.txt', names=['src', 'tar', 'lic'], sep='\t')
del lines['lic']
print('전체 샘플의 개수 :',len(lines))

전체 샘플의 개수 : 232736


다운로드한 Parallel corpus 샘플 중 6만개만 사용  
\+ (Colab 램 제한 때문에 2만개로 줄임)

In [13]:
lines = lines.loc[:, 'src':'tar']
lines = lines[0:20000] # 6만개만 저장
lines.sample(10)

,src,tar
17114,Pour me a drink.,Versez-moi un verre !
4391,We eat meat.,Nous mangeons de la viande.
13589,We all cheered.,Nous acclamèrent tous.
17996,Tom is studying.,Tom est en train d'étudier.
5145,I called you.,Je vous ai appelée.
11275,I feel trapped.,Je me sens piégée.
7186,Be reasonable.,Soyez raisonnables !
4282,Tom shut up.,Tom a fermé sa gueule.
6996,You're dirty.,Tu es sale.
16161,I'm at my place.,Je suis chez moi.


## 학습 데이터 전처리

프랑스어 데이터에 \<sos>, \<eos>를 대신할 '\t', '\n' 삽입

In [14]:
lines.tar = lines.tar.apply(lambda x : '\t '+ x + ' \n')
lines.sample(10)

,src,tar
2009,I can't go.,\t Je ne peux pas partir. \n
18191,Tom was on time.,\t Tom était à l'heure. \n
8310,I'll watch TV.,\t Je vais regarder la télé. \n
7435,Get your gear.,\t Va chercher ton matériel ! \n
13891,Where's my bag?,\t Où est ma bourse ? \n
14665,Do what you can.,\t Fais ce que tu peux. \n
5137,I ate caviar.,\t J'ai mangé du caviar. \n
12940,The siren blew.,\t La sirène a retenti. \n
15494,I have leukemia.,\t J’ai une leucémie. \n
9556,Tom is family.,\t Tom fait partie de la famille. \n


영어, 프랑스어 문자 집합을 구축하고,
각 집합별로 문자 단위로 index 부여

In [15]:
# 문자 집합 구축
src_vocab = set()
for line in lines.src: # 1줄씩 읽음
    for char in line: # 1개의 문자씩 읽음
        src_vocab.add(char)

tar_vocab = set()
for line in lines.tar:
    for char in line:
        tar_vocab.add(char)

# char set size 확인
src_vocab_size = len(src_vocab)+1
tar_vocab_size = len(tar_vocab)+1
print('source 문장의 char 집합 :',src_vocab_size)
print('target 문장의 char 집합 :',tar_vocab_size)

# index 부여 전 정렬
src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))

# 각 문자에 index 부여
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])

# 영어, 프랑스어 (source, target) 각각의 char set index 부여 확인
print(src_to_index)
print(tar_to_index)

source 문장의 char 집합 : 75
target 문장의 char 집합 : 100
{' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, '&': 6, "'": 7, ',': 8, '-': 9, '.': 10, '/': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ':': 22, '?': 23, 'A': 24, 'B': 25, 'C': 26, 'D': 27, 'E': 28, 'F': 29, 'G': 30, 'H': 31, 'I': 32, 'J': 33, 'K': 34, 'L': 35, 'M': 36, 'N': 37, 'O': 38, 'P': 39, 'Q': 40, 'R': 41, 'S': 42, 'T': 43, 'U': 44, 'V': 45, 'W': 46, 'Y': 47, 'a': 48, 'b': 49, 'c': 50, 'd': 51, 'e': 52, 'f': 53, 'g': 54, 'h': 55, 'i': 56, 'j': 57, 'k': 58, 'l': 59, 'm': 60, 'n': 61, 'o': 62, 'p': 63, 'q': 64, 'r': 65, 's': 66, 't': 67, 'u': 68, 'v': 69, 'w': 70, 'x': 71, 'y': 72, 'z': 73, 'é': 74}
{'\t': 1, '\n': 2, ' ': 3, '!': 4, '$': 5, '%': 6, '&': 7, "'": 8, ',': 9, '-': 10, '.': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ':': 22, '?': 23, 'A': 24, 'B': 25, 'C': 26, 'D': 27, 'E': 28, 'F': 29, 'G': 30, 'H': 31, 'I': 32, 'J': 33, 'K

indexing된 character set을 이용하여, training data를 정수형으로 encoding.  
(encoder의 input이 될 영어 문장과, decoder의 input이 될 프랑스 문장에 대해)

In [16]:
encoder_input = []

# 1개의 문장
for line in lines.src:
  encoded_line = []
  # 각 줄에서 1개의 char
  for char in line:
    # 각 char을 정수로 변환
    encoded_line.append(src_to_index[char])
  encoder_input.append(encoded_line)
print('source 문장의 정수 인코딩 :',encoder_input[:5])

decoder_input = []
for line in lines.tar:
  encoded_line = []
  for char in line:
    encoded_line.append(tar_to_index[char])
  decoder_input.append(encoded_line)
print('target 문장의 정수 인코딩 :',decoder_input[:5])

source 문장의 정수 인코딩 : [[30, 62, 10], [30, 62, 10], [30, 62, 10], [30, 62, 10], [31, 56, 10]]
target 문장의 정수 인코딩 : [[1, 3, 45, 48, 3, 4, 3, 2], [1, 3, 36, 48, 65, 50, 55, 52, 11, 3, 2], [1, 3, 28, 61, 3, 65, 62, 68, 67, 52, 3, 4, 3, 2], [1, 3, 25, 62, 68, 54, 52, 3, 4, 3, 2], [1, 3, 42, 48, 59, 68, 67, 3, 4, 3, 2]]


또한, decoder의 output과 비교할 프랑스어 실제값을 encoding.  
단, true value는 \<sos>가 필요하지 않으므로 제거하고 encoding.

In [17]:
decoder_target = []
for line in lines.tar:
  timestep = 0
  encoded_line = []
  for char in line:
    if timestep > 0:
      encoded_line.append(tar_to_index[char])
    timestep = timestep + 1
  decoder_target.append(encoded_line)
print('target 문장 레이블의 정수 인코딩 :',decoder_target[:5])

target 문장 레이블의 정수 인코딩 : [[3, 45, 48, 3, 4, 3, 2], [3, 36, 48, 65, 50, 55, 52, 11, 3, 2], [3, 28, 61, 3, 65, 62, 68, 67, 52, 3, 4, 3, 2], [3, 25, 62, 68, 54, 52, 3, 4, 3, 2], [3, 42, 48, 59, 68, 67, 3, 4, 3, 2]]


영어 데이터와 프랑스어 데이터에 대해, 각각의 데이터마다 가장 긴 문장에 맞추어 padding.  
이후, 모든 값에 대해 one-hot encoding 수행.  

이 예시에서는 character 단위의 prediction을 수행하므로, word embedding을 수행하지 않음.

In [18]:
max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])
print('source 문장의 최대 길이 :',max_src_len)
print('target 문장의 최대 길이 :',max_tar_len)

# 가장 긴 문장에 맞추어 padding
encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')

# one-hot encoding
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

source 문장의 최대 길이 : 17
target 문장의 최대 길이 : 61


# Seq2Seq 모델 설계 및 Training

In [19]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
import numpy as np

encoder_states: context vector. (last hidden state & last cell state of encoder)

In [20]:
encoder_inputs = Input(shape=(None, src_vocab_size))
encoder_lstm = LSTM(units=256, return_state=True)

# encoder_outputs은 여기서는 불필요
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

# LSTM은 바닐라 RNN과는 달리 상태가 두 개. 은닉 상태와 셀 상태.
encoder_states = [state_h, state_c]

Decoder에 context vector(`encoder_states`)를 첫 hidden state, cell state로 사용.

output 도출 시 Dense - softmax를 거치도록 설정.

optimizer는 RMSProp, loss func은 CCE.

In [21]:
decoder_inputs = Input(shape=(None, tar_vocab_size))
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)

# 디코더에게 인코더의 은닉 상태, 셀 상태를 전달.
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state=encoder_states)

decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

학습.

In [22]:
model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=64, epochs=40, validation_split=0.2)

Epoch 1/40
250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - loss: 1.5027 - val_loss: 1.0122
Epoch 2/40
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 0.8277 - val_loss: 0.8089
Epoch 3/40
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - loss: 0.6793 - val_loss: 0.7357
Epoch 4/40
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 0.6162 - val_loss: 0.6898
Epoch 5/40
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - loss: 0.5790 - val_loss: 0.6574
Epoch 6/40
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - loss: 0.5518 - val_loss: 0.6317
Epoch 7/40
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - loss: 0.5262 - val_loss: 0.6061
Epoch 8/40
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - loss: 0.5058 - val_loss: 0.5856
Epoch 9/40
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - loss: 0.4850 - val_loss: 0.5660
Epoch 10/40
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 0.4682 - val_loss: 0.5492
Epoch 11/40
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - loss: 0.4531 - val_loss: 0.5338
Epoch 12/40
250/250 ━━━━━━━━━━━━━━━━━━━━ 

# 학습된 모델을 Test

In [23]:
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

# 이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용.
# 뒤의 함수 decode_sequence()에 동작을 구현 예정
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)

# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태를 버리지 않음.
decoder_states = [state_h, state_c]
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)

In [24]:
# index -> 단어 함수
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())

In [25]:
def decode_sequence(input_seq):
  # 입력으로부터 인코더의 상태를 얻음
  states_value = encoder_model.predict(input_seq)

  # <SOS>에 해당하는 원-핫 벡터 생성
  target_seq = np.zeros((1, 1, tar_vocab_size))
  target_seq[0, 0, tar_to_index['\t']] = 1.

  stop_condition = False
  decoded_sentence = ""

  # stop_condition이 True가 될 때까지 루프 반복
  while not stop_condition:
    # 이전 시점의 상태 states_value를 현 시점의 초기 상태로 사용
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

    # 예측 결과를 문자로 변환
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = index_to_tar[sampled_token_index]

    # 현재 시점의 예측 문자를 예측 문장에 추가
    decoded_sentence += sampled_char

    # <eos>에 도달하거나 최대 길이를 넘으면 중단.
    if (sampled_char == '\n' or
        len(decoded_sentence) > max_tar_len):
        stop_condition = True

    # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, sampled_token_index] = 1.

    # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
    states_value = [h, c]

  return decoded_sentence

In [26]:
for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스
  input_seq = encoder_input[seq_index:seq_index+1]
  decoded_sentence = decode_sequence(input_seq)
  print(35 * "-")
  print('입력 문장:', lines.src[seq_index])
  print('정답 문장:', lines.tar[seq_index][2:len(lines.tar[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
  print('번역 문장:', decoded_sentence[1:len(decoded_sentence)-1]) # '\n'을 빼고 출력

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 283ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
-----------------------------------
입력 문장: Go.
정답 문장: Bouge ! 
번역 문장: Regardez ! 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 